In [1]:
import pickle

import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
!pip install --upgrade keras

import keras

print(keras.__version__)

3.9.1


In [4]:
## Load the dataset
data = pd.read_csv(
    "/Users/sunnythesage/PythonProjects/Data-Science-BootCamp/03-Deep-Learning-BootCamp/7 - End to End Deep Learning Project Using ANN/advanced-customer-churn-analysis-using-ann/data/raw/churn-modelling-dataset.csv")

data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Data Preprocessing

In [5]:
# rop irrelevant columns

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [6]:
## Encode categorical variables

label_encoder_gender = LabelEncoder()

data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
## Onehot encode Geography column
from sklearn.preprocessing import OneHotEncoder

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [8]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
geo_encoded_df = pd.DataFrame(geo_encoder, columns = onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
## Combine one hot encoder columns with the original data
data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
# Save encoders and scaler for later use

import os

# Define the project's root directory
project_root = "/Users/sunnythesage/PythonProjects/Data-Science-BootCamp/03-Deep-Learning-BootCamp/7 - End to End Deep Learning Project Using ANN/advanced-customer-churn-analysis-using-ann"

# Change the current working directory to the project's root
os.chdir(project_root)

# --- Artifacts ---

# Define the relative path to the artifacts directory
artifacts_dir = os.path.join(os.getcwd(), 'artifacts')

# Create the directory if it doesn't exist
os.makedirs(artifacts_dir, exist_ok = True)

print(artifacts_dir)

/Users/sunnythesage/PythonProjects/Data-Science-BootCamp/03-Deep-Learning-BootCamp/7 - End to End Deep Learning Project Using ANN/advanced-customer-churn-analysis-using-ann/artifacts


In [12]:
# Save encoders and scaler to the artifacts folder
with open(os.path.join(artifacts_dir, 'label_encoder_gender.pkl'), 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open(os.path.join(artifacts_dir, 'onehot_encoder_geo.pkl'), 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [13]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [14]:
## Divide the dataset into independent and dependent features
X = data.drop('Exited', axis = 1)
y = data['Exited']

## Split the data in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [16]:
with open(os.path.join(artifacts_dir, 'scaler.pkl'), 'wb') as file:
    pickle.dump(scaler, file)

In [17]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


### ANN Implementation

In [18]:
from keras import layers

In [19]:
from keras import Sequential

In [20]:
(X_train.shape[1],)

(12,)

In [21]:
# Build Our ANN Model
model = Sequential([
    layers.Input(shape = (X_train.shape[1],)),  # Explicitly define the input shape
    layers.Dense(64, activation = 'relu'),  # HL1
    layers.Dense(32, activation = 'relu'),  # HL2
    layers.Dense(1, activation = 'sigmoid')  # Output layer
])

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
opt = keras.optimizers.Adam(learning_rate = 0.01)
loss = keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x302209440>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [24]:
## compile the model
model.compile(optimizer = opt, loss = "binary_crossentropy", metrics = ['accuracy'])

In [25]:
## Set up the Tensorboard

import datetime

# --- Logs ---

# Define the root log directory (similar to artifacts)
root_log_dir = os.path.join(os.getcwd(), 'logs')

# Create the root log directory if it doesn't exist
os.makedirs(root_log_dir, exist_ok = True)

# Generate a unique log directory for each run
log_dir = os.path.join(root_log_dir, "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

# Create the specific log directory for this run
os.makedirs(log_dir, exist_ok = True)

print(root_log_dir)
print(log_dir)

/Users/sunnythesage/PythonProjects/Data-Science-BootCamp/03-Deep-Learning-BootCamp/7 - End to End Deep Learning Project Using ANN/advanced-customer-churn-analysis-using-ann/logs
/Users/sunnythesage/PythonProjects/Data-Science-BootCamp/03-Deep-Learning-BootCamp/7 - End to End Deep Learning Project Using ANN/advanced-customer-churn-analysis-using-ann/logs/fit/20250327-234536


In [26]:
# Create the TensorBoard callback
tensorflow_callback = keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [27]:
## Set up Early Stopping
early_stopping_callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10,
                                                        restore_best_weights = True)


In [28]:
### Train the model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8084 - loss: 0.4393 - val_accuracy: 0.8595 - val_loss: 0.3471
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step - accuracy: 0.8571 - loss: 0.3568 - val_accuracy: 0.8630 - val_loss: 0.3429
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step - accuracy: 0.8526 - loss: 0.3489 - val_accuracy: 0.8625 - val_loss: 0.3413
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - accuracy: 0.8608 - loss: 0.3397 - val_accuracy: 0.8585 - val_loss: 0.3402
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step - accuracy: 0.8676 - loss: 0.3272 - val_accuracy: 0.8595 - val_loss: 0.3432
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - accuracy: 0.8646 - loss: 0.3325 - val_accuracy: 0.8630 - val_loss: 0.3369
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step - accuracy: 0.8649 - loss: 0.3355 - val_accuracy: 0.8590 - val_loss: 0.3545
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - accuracy: 0.8608 - loss: 0.3

In [29]:
# model.save('model.keras')

keras.saving.save_model(model, os.path.join(artifacts_dir, 'model.keras'))

In [30]:
## Load Tensorboard Extension
%load_ext tensorboard

In [31]:
%tensorboard --logdir logs/fit